<hr>
<div style="background-color: lightgray; padding: 20px; color: black;">
<div>
<img src="https://th.bing.com/th/id/R.3cd1c8dc996c5616cf6e65e20b6bf586?rik=09aaLyk4hfbBiQ&riu=http%3a%2f%2fcidics.uanl.mx%2fwp-content%2fuploads%2f2016%2f09%2fcimat.png&ehk=%2b0brgMUkA2BND22ixwLZheQrrOoYLO3o5cMRqsBOrlY%3d&risl=&pid=ImgRaw&r=0" style="float: right; margin-right: 30px;" width="200"/> 
<font size="5.5" color="8C3061"><b>Only-Encoder Transformer para predicción de series temporales </b></font> <br>
<font size="4.5" color="8C3061"><b>Aprendizaje de Máquina II - Tarea 2 </b></font> 
</div>
<div style="text-align: left">  <br>
Edison David Serrano Cárdenas. <br>
MSc en Matemáticas Aplicadas <br>
CIMAT - Sede Guanajuato <br>
</div>

</div>
<hr>


## <font color="8C3061" >**Cargar Librerías**</font> 

**Cargar Librerías:**

In [1]:
import numpy as np
import pandas as pd
import random, math
import yfinance as yf

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)  # Activar el modo de cuaderno
sns.set_theme()

import torch
from torch import nn, Tensor
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from seq2seq import Encoder, Decoder, Seq2Seq
from seq2seq import train_seq2seq


**Verificación de CUDA, Fijar la Semilla**

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device is:",device)

SEED = 42
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(SEED)

device is: cuda


## <font color="8C3061" >**Enunciado del problema**</font> 

Considere el valor histórico por hora de cripto-monedas de al menos 7 series, incluyendo el bitcoin, obtenga N series de al menos una longitud de T = 100. 

Luego entrene un modelo para dadas las series S[:T-t] prediga la parte final de cada serie S[T-t:]. Un valor típico es t=5.

Los datos descárgelos usando la librería yahoo-finance 

## <font color="8C3061" >**Preprocesamiento de los Datos**</font> 

Descarga de los historicos de cada criptomoneda usando la librería yfinance

In [3]:
# Lista de criptomonedas
cryptos = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD', 'ADA-USD', 'SOL-USD', 'DOGE-USD']

# Descargar datos por hora
def download_crypto_data(tickers, period='1mo', interval='1h'):
    data = {}
    for ticker in tickers:
        crypto_data = yf.download(ticker, period=period, interval=interval)
        crypto_data['Volume'] = crypto_data['Volume'].astype(float)
        data[ticker] = crypto_data[['Open', 'High', 'Low', 'Close', 'Volume']]
        
    return data

# Descargar los datos de las criptomonedas
crypto_data = download_crypto_data(cryptos)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Normalización de los datos:

In [4]:
def normalize_data_zscore(data):
    normalized_data = {}
    stats = {}  # Para almacenar la media y la desviación estándar de cada criptomoneda
    for ticker, df2 in data.items():
        df = df2.copy(deep=True)
        means = df[['Open', 'High', 'Low', 'Close', 'Volume']].mean()
        stds = df[['Open', 'High', 'Low', 'Close', 'Volume']].std()
        stats[ticker] = {'mean': means, 'std': stds}
        df.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume']] = (df[['Open', 'High', 'Low', 'Close', 'Volume']] - means) / stds
        normalized_data[ticker] = df
    return normalized_data, stats

def normalize_data_minmax(data_original):
    data = data_original.copy()
    normalized_data = {}
    scalers = {}  # Para almacenar el MinMaxScaler de cada criptomoneda
    for ticker, df2 in data.items():
        df = df2.copy(deep=True)
        scaler = MinMaxScaler()
        df.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume']] = scaler.fit_transform(df[['Open', 'High', 'Low', 'Close', 'Volume']])
        scalers[ticker] = scaler
        normalized_data[ticker] = df
    return normalized_data, scalers

def normalize_data_log(data_original):
    data = data_original.copy()
    normalized_data = {}
    for ticker, df2 in data.items():
        df = df2.copy(deep=True)
        df.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume']] = np.log1p(df[['Open', 'High', 'Low', 'Close', 'Volume']])
        normalized_data[ticker] = df
    return normalized_data

Desnormalización de los datos:

In [5]:
def denormalize_close_value_zscore(ticker, normalized_value, stats):
    mean_close = stats[ticker]['mean']['Close']
    std_close = stats[ticker]['std']['Close']
    return normalized_value * std_close + mean_close

def denormalize_close_value_minmax(ticker, normalized_close_value, scalers):
    place_holder = np.zeros((normalized_close_value.shape[0],5))
    place_holder[:,3]= normalized_close_value
    
    return scalers[ticker].inverse_transform(place_holder)[:,3]

def denormalize_close_value_log(normalized_value):
    return np.expm1(normalized_value)

Creación de la librería CryptoDataset:

In [6]:
class CryptoDataset(Dataset):
    def __init__(self, data, t=5):
        self.data = []
        self.train = []     
        self.test = []
        self.t = t
        
        for ticker, df in data.items():
            series = df[['Open', 'High', 'Low', 'Close', 'Volume']].values
            l = len(series)
            train_size = int(0.8*l)
            
            for i in range(l - 100):
                input_seq = series[i:i+(100-t)]  # Últimos 100-t pasos como entrada
                target_seq = series[i+(100-t):i+100, 3]  # Solo el precio de cierre ('Close') como salida
                
                self.data.append((input_seq, target_seq))
                
                if (i<train_size):
                    self.train.append((input_seq, target_seq))
                else:
                    self.test.append((input_seq, target_seq))
                
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.FloatTensor(x), torch.FloatTensor(y).unsqueeze(1)  # y debe ser de la forma (batch_size, 1)

Datos Preprocesados:

In [7]:
# Crear el dataset
normalized_crypto_data_zscore, crypto_stats = normalize_data_zscore(crypto_data)
normalized_crypto_data_minmax, crypto_scaler = normalize_data_minmax(crypto_data)
normalized_crypto_data_log = normalize_data_log(crypto_data)

only_enc_model_zscore = CryptoDataset(normalized_crypto_data_zscore)
only_enc_model_min_max = CryptoDataset(normalized_crypto_data_minmax)
only_enc_model_log = CryptoDataset(normalized_crypto_data_log)

# Dividir en training y test
train_dataset_zscore, test_dataset_zscore = only_enc_model_zscore.train, only_enc_model_zscore.test
train_dataset, test_dataset = only_enc_model_min_max.train, only_enc_model_min_max.test
train_dataset_log, test_dataset_log = only_enc_model_log.train, only_enc_model_log.test

In [8]:
print("Número de ejemplos de entrenamiento:\t", len(train_dataset)," y ", len(train_dataset_log))
print("Número de ejemplos de test:\t\t", len(test_dataset)," y ", len(test_dataset_log))

Número de ejemplos de entrenamiento:	 4032  y  4032
Número de ejemplos de test:		 315  y  315


In [9]:
# Crear DataLoaders
BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

## <font color="8C3061" >**Definición del Modelo**</font> 

### <font color="8C3061" >**Positional Encoding**</font> 

In [10]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

### <font color="8C3061" >**ForecastingModel**</font> 

Implementación del modelo (OnlyEncoder Transformer) para Predicción de Series Temporales 

In [11]:
class ForecastingModel(nn.Module):
    def __init__(self, 
                 seq_len=95,  # Ajuste para tus secuencias de 100 pasos
                 pred_len=5,
                 embed_size=16,
                 nhead=4,
                 dim_feedforward=2048,
                 dropout=0.1,
                 conv1d_emb=True,
                 conv1d_kernel_size=3,
                 device="cuda"):
        super(ForecastingModel, self).__init__()
        
        # Fijar los parámetros
        self.device = device
        self.conv1d_emb = conv1d_emb
        self.conv1d_kernel_size = conv1d_kernel_size
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.embed_size = embed_size
        
        # Capa de embedding
        if conv1d_emb:
            if conv1d_kernel_size % 2 == 0:
                raise Exception("conv1d_kernel_size must be an odd number to preserve dimensions.")
            self.conv1d_padding = conv1d_kernel_size - 1
            self.input_embedding = nn.Conv1d(5, 
                                             embed_size, 
                                             kernel_size=conv1d_kernel_size)  # 5 características de entrada
        else: 
            self.input_embedding = nn.Linear(5, 
                                             embed_size)  # 5 características de entrada

        # Capa de positional encoding
        self.position_encoder = PositionalEncoding(d_model=embed_size, 
                                                   dropout=dropout, 
                                                   max_len=seq_len)
        
        # Capa de Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_size, 
                                                   nhead=nhead, 
                                                   dim_feedforward=dim_feedforward, 
                                                   dropout=dropout,
                                                   batch_first=True)
        # Se define el número de capas en el modelo de Transformer
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, 
                                                         num_layers=3)
        
        # Componente de Regresión
        self.linear1 = nn.Linear(seq_len * embed_size, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, int(dim_feedforward / 2))
        self.linear3 = nn.Linear(int(dim_feedforward / 2), int(dim_feedforward / 4))
        self.linear4 = nn.Linear(int(dim_feedforward / 4), self.pred_len)  # Predict 5 values directly


        # Elementos básicos
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x: Tensor) -> Tensor:
        src_mask = self._generate_square_subsequent_mask()
        src_mask = src_mask.to(self.device)
        batch_size = x.size(0)
        
        # Embedding
        if self.conv1d_emb:
            x = F.pad(x, (0, 0, self.conv1d_padding, 0), "constant", -1)
            x = self.input_embedding(x.transpose(1, 2))
            x = x.transpose(1, 2)
        else:
            x = self.input_embedding(x)
        
        # Positional Encoding
        x = self.position_encoder(x)
        
        # Transformer Encoder
        x = self.transformer_encoder(x, mask=src_mask)
        
        # Flatten para la capa de regresión
        x = x.reshape(batch_size, -1)
        
        # Regresión
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # Salida
        x = self.linear4(x)     
        
        return x
        
    # Function to create source mask
    def _generate_square_subsequent_mask(self):
        return torch.triu(torch.full((self.seq_len, self.seq_len), float('-inf'), device=self.device), diagonal=1)

## <font color="8C3061" >**Entrenamiento del Modelo**</font> 

In [12]:
# Definir hiperparámetros
d_model = 64
nhead = 4
num_encoder_layers = 3
dim_feedforward =  2048
dropout = 0.1

In [13]:
# Función de entrenamiento
def train_only_encoder(model, dataloader, optimizer, criterion, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for x_batch, y_batch in dataloader:
            optimizer.zero_grad()
            
            # Convertir los datos a float32 y enviarlos al dispositivo
            x_batch = x_batch.to(device).float()
            y_batch = y_batch.to(device).float()
            
            # Forward pass
            output = model(x_batch)
            
            # Calcular la pérdida
            loss = criterion(output, y_batch)
            total_loss += loss.item()
            
            # Backward pass y optimización
            loss.backward()
            optimizer.step()
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader)}')

### <font color="8C3061" >**Entrenamiento con Normalización MinMaxScaler**</font> 

In [14]:
# Inicializar el modelo
only_encoder_model_min_max = ForecastingModel(seq_len=95,
                                        pred_len=5,
                                        embed_size=d_model,
                                        nhead=nhead,
                                        dim_feedforward=dim_feedforward,
                                        dropout=dropout,
                                        conv1d_emb=True,
                                        conv1d_kernel_size=3,
                                        device=device)

# Mover el modelo al dispositivo
only_encoder_model_min_max = only_encoder_model_min_max.to(device)

# Definir el optimizador y la función de pérdida
optimizer = optim.Adam(only_encoder_model_min_max.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Entrenar el modelo
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
train_only_encoder(only_encoder_model_min_max, train_loader, optimizer, criterion, num_epochs=10)

Epoch 1/10, Loss: 0.2869770136617479
Epoch 2/10, Loss: 0.019744253732145778
Epoch 3/10, Loss: 0.01006031322795602
Epoch 4/10, Loss: 0.00923590248798035
Epoch 5/10, Loss: 0.0097332744496978
Epoch 6/10, Loss: 0.007585691953344005
Epoch 7/10, Loss: 0.007123799078048222
Epoch 8/10, Loss: 0.00757422122646064
Epoch 9/10, Loss: 0.007773471504656805
Epoch 10/10, Loss: 0.007742570251196859


### <font color="8C3061" >**Entrenamiento con Normalización LogScale**</font> 

In [15]:
only_encoder_model_log = ForecastingModel(seq_len=95,
                                        pred_len=5,
                                        embed_size=d_model,
                                        nhead=nhead,
                                        dim_feedforward=dim_feedforward,
                                        dropout=dropout,
                                        conv1d_emb=True,
                                        conv1d_kernel_size=3,
                                        device=device)

# Mover el modelo al dispositivo
only_encoder_model_log = only_encoder_model_log.to(device)

# Definir el optimizador y la función de pérdida
optimizer = optim.Adam(only_encoder_model_log.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Entrenar el modelo
train_loader = DataLoader(train_dataset_log, batch_size=32, shuffle=True)
train_only_encoder(only_encoder_model_log, train_loader, optimizer, criterion, num_epochs=10)

Epoch 1/10, Loss: 6.156117548308675
Epoch 2/10, Loss: 0.6389724065860113
Epoch 3/10, Loss: 0.4079183455970552
Epoch 4/10, Loss: 0.44113040153705885
Epoch 5/10, Loss: 0.3853672154484287
Epoch 6/10, Loss: 0.43340905471926644
Epoch 7/10, Loss: 0.34676383808255196
Epoch 8/10, Loss: 0.36015538599282976
Epoch 9/10, Loss: 0.2885715601936219
Epoch 10/10, Loss: 0.33815511126839926


### <font color="8C3061" >**Entrenamiento con Normalización Zscore**</font> 

In [16]:
only_enc_model_zscore = ForecastingModel(seq_len=95,
                                        pred_len=5,
                                        embed_size=d_model,
                                        nhead=nhead,
                                        dim_feedforward=dim_feedforward,
                                        dropout=dropout,
                                        conv1d_emb=True,
                                        conv1d_kernel_size=3,
                                        device=device)

# Mover el modelo al dispositivo
only_enc_model_zscore = only_enc_model_zscore.to(device)

# Definir el optimizador y la función de pérdida
optimizer = optim.Adam(only_enc_model_zscore.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Entrenar el modelo
train_loader = DataLoader(train_dataset_zscore, batch_size=32, shuffle=True)
train_only_encoder(only_enc_model_zscore, train_loader, optimizer, criterion, num_epochs=10)

Epoch 1/10, Loss: 0.2483365736192181
Epoch 2/10, Loss: 0.08601539985587199
Epoch 3/10, Loss: 0.07042444943790398
Epoch 4/10, Loss: 0.056120845873559276
Epoch 5/10, Loss: 0.05095215104815979
Epoch 6/10, Loss: 0.05099749174856004
Epoch 7/10, Loss: 0.04832419559418682
Epoch 8/10, Loss: 0.05209470813768723
Epoch 9/10, Loss: 0.04131156021344756
Epoch 10/10, Loss: 0.04239488636986131


### <font color="8C3061" >**Entrenamiento del modelo Seq2Seq con Atención**</font> 

In [17]:
from seq2seq import Encoder, Decoder, Seq2Seq
from seq2seq import train_seq2seq

In [18]:
encoder = Encoder(input_size=5, hidden_size=64).to(device)
decoder = Decoder(output_size=1, hidden_size=64).to(device)
seq2seq_model = Seq2Seq(encoder, decoder, device).to(device)

# Optimizer y Loss
optimizer = torch.optim.Adam(seq2seq_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

train_loader = DataLoader(train_dataset_zscore, batch_size=32, shuffle=True)
train_seq2seq(seq2seq_model, train_loader, optimizer, criterion, num_epochs=10)

/media/edserranoc/Data/EDSC_2024/Code/CIMAT/Semestre3/Machine_Learning2/mlenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:88: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.35 and num_layers=1



Epoch 1/10, Loss: 0.060207199305295944
Epoch 2/10, Loss: 0.027384642511606216
Epoch 3/10, Loss: 0.015301431529223919
Epoch 4/10, Loss: 0.021142031997442245
Epoch 5/10, Loss: 0.016975609585642815
Epoch 6/10, Loss: 0.010969390161335468
Epoch 7/10, Loss: 0.02579141966998577
Epoch 8/10, Loss: 0.018433166667819023
Epoch 9/10, Loss: 0.022065868601202965
Epoch 10/10, Loss: 0.014606550335884094


## <font color="8C3061" >**Evaluación del Modelo**</font>

Las métricas de evaluación son el error cuadrático medio $(MSE)$, el error absoluto medio $(MAE)$ y el coeficiente de determinación $(R^2)$

In [19]:
test_loader_zscore = DataLoader(test_dataset_zscore, batch_size=1, shuffle=False)
test_loader_log = DataLoader(test_dataset_log, batch_size=1, shuffle=False)
test_loader_min_max = DataLoader(test_dataset, batch_size=1, shuffle=False)

Evaluación del rendimiento

In [20]:
def test_evaluation(model, test_loader,name_model = None):
    predicted_close_values = []
    true_close_values = []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.to(device).float()
            y_batch = y_batch.to(device).float()
            
            output = model(x_batch)
            
            predicted_close_values.extend(output.cpu().numpy())
            true_close_values.extend(y_batch.cpu().numpy())
            
    true_close_values = np.array(true_close_values)
    predicted_close_values = np.array(predicted_close_values)
    
    if name_model == "seq2seq":
        predicted_close_values = predicted_close_values[:,:,0]
    
    mse = mean_squared_error(true_close_values, predicted_close_values)
    mae = mean_absolute_error(true_close_values, predicted_close_values)
    r2 = r2_score(true_close_values, predicted_close_values)
    
    return mse, mae, r2

In [21]:
score = pd.DataFrame(columns=["Model","MSE","MAE","R2"])

mse, mae, r2 = test_evaluation(only_enc_model_zscore, test_loader_zscore)
score.loc[0] = ["transf_encoder_zscore",mse,mae,r2]

seq2seq_model.eval()
mse, mae, r2 = test_evaluation(seq2seq_model, test_loader_zscore,name_model = "seq2seq")
score.loc[1] = ["seq2seq",mse,mae,r2]

mse, mae, r2 = test_evaluation(only_encoder_model_min_max, test_loader_min_max)
score.loc[2] = ["transf_encoder_min_max",mse,mae,r2]

mse, mae, r2 = test_evaluation(only_encoder_model_log, test_loader_log)
score.loc[3] = ["transf_encoder_log",mse,mae,r2]

Rendimiento de los modelos

In [22]:
def highlight_min(s):
    return ['background-color: red' if v == s.min() else '' for v in s]

def highlight_max(s):
    return ['background-color: red' if v == s.max() else '' for v in s]

score.style.apply(highlight_min, subset=['MSE', 'MAE']).apply(highlight_max, subset=['R2'])

,Model,MSE,MAE,R2
0,transf_encoder_zscore,0.145145,0.276519,0.493075
1,seq2seq,0.062212,0.176201,0.782130
2,transf_encoder_min_max,0.031238,0.146365,-1.066178
3,transf_encoder_log,0.390266,0.429487,0.975332


## <font color="8C3061" >**Inferencia de la Criptomoneda BitCoin**</font> 

Inferir valores a partir de una serie temporal:

In [23]:
def inference(model, 
             data, 
             crypto_stats =None, 
             scaler=None, 
             name_model = None,
             coin = "BTC-USD"
             ):
    
    predicted_close_values = []
    true_close_values = []
    with torch.no_grad():
        for x_batch, y_batch in data:
            x_batch = x_batch.to(device).float()
            y_batch = y_batch.to(device).float()
            
            output = model(x_batch)
            
            predicted_close_values.extend(output.cpu().numpy())
            true_close_values.extend(y_batch.cpu().numpy())
            
    true_close_values = np.array(true_close_values)
    predicted_close_values = np.array(predicted_close_values)
    
    if name_model == "seq2seq":
        predicted_close_values = predicted_close_values[:,:,0]
    
    if crypto_stats:
        predicted_close_values = denormalize_close_value_zscore(coin, predicted_close_values, crypto_stats)
        true_close_values = denormalize_close_value_zscore(coin, true_close_values, crypto_stats)
    elif scaler:
        predicted_close_values = denormalize_close_value_minmax(coin, predicted_close_values[0], scaler)
        true_close_values = denormalize_close_value_minmax(coin, true_close_values[0].T[0], scaler)
    else:
        predicted_close_values = denormalize_close_value_log(predicted_close_values)
        true_close_values = denormalize_close_value_log(true_close_values)
    
    return true_close_values, predicted_close_values

In [24]:
def inference_results(coin="BTC-USD"):
    coin_x_zscore = normalized_crypto_data_zscore[coin][-100:-5][['Open', 'High', 'Low', 'Close', 'Volume']].values
    coin_y_zscore = normalized_crypto_data_zscore[coin][-5:][['Close']].values
    
    coin_x_min_max = normalized_crypto_data_minmax[coin][-100:-5][['Open', 'High', 'Low', 'Close', 'Volume']].values
    coin_y_min_max = normalized_crypto_data_minmax[coin][-5:][['Close']].values
    
    coin_x_log = normalized_crypto_data_log[coin][-100:-5][['Open', 'High', 'Low', 'Close', 'Volume']].values
    coin_y_log = normalized_crypto_data_log[coin][-5:][['Close']].values
    
    coin_data_zscore = DataLoader([(torch.FloatTensor(coin_x_zscore), torch.FloatTensor(coin_y_zscore))], batch_size=1, shuffle=False)
    coin_data_min_max = DataLoader([(torch.FloatTensor(coin_x_min_max), torch.FloatTensor(coin_y_min_max))], batch_size=1, shuffle=False)
    coin_data_log = DataLoader([(torch.FloatTensor(coin_x_log), torch.FloatTensor(coin_y_log))], batch_size=1, shuffle=False)
    
    inference_dict = {}
    
    true_close_values, predicted_close_values = inference(only_enc_model_zscore, coin_data_zscore, crypto_stats=crypto_stats)
    inference_dict["transf_encoder_zscore"] = {"true": true_close_values, "predicted": predicted_close_values}   
    
    true_close_values, predicted_close_values = inference(seq2seq_model, coin_data_zscore, name_model = "seq2seq", crypto_stats=crypto_stats)
    inference_dict["seq2seq"] = {"true": true_close_values, "predicted": predicted_close_values}
    
    true_close_values, predicted_close_values = inference(only_encoder_model_log, coin_data_log)
    inference_dict["transf_encoder_log"] = {"true": true_close_values, "predicted": predicted_close_values}
    
    true_close_values, predicted_close_values = inference(only_encoder_model_min_max, coin_data_min_max, scaler=crypto_scaler)
    inference_dict["transf_encoder_min_max"] = {"true": true_close_values, "predicted": predicted_close_values}
    
    time_data = normalized_crypto_data_zscore[coin][-5:].index
    
    return inference_dict,time_data

In [25]:
inference_dict,time_data = inference_results(coin="BTC-USD")
constant_value = crypto_data['BTC-USD']['Close'].iloc[-6]

In [33]:
fig = go.Figure()

# Add real time series trace
fig.add_trace(go.Scatter(x=time_data, y=inference_dict["transf_encoder_zscore"]["true"].flatten(), 
                         mode='lines+markers', name='Valores reales'))

# Add predicted values trace for each model
fig.add_trace(go.Scatter(x=time_data, y=inference_dict["transf_encoder_zscore"]["predicted"].flatten(), 
                         mode='lines+markers', name='Predicción transf_encoder_zscore'))

fig.add_trace(go.Scatter(x=time_data, y=inference_dict["seq2seq"]["predicted"].flatten(), 
                         mode='lines+markers', name='Predicción seq2seq'))

#fig.add_trace(go.Scatter(x=time_data, y=inference_dict["transf_encoder_log"]["predicted"].flatten(), #
#                         mode='lines+markers', name='Predicción transf_encoder_log'))

fig.add_trace(go.Scatter(x=time_data, y=inference_dict["transf_encoder_min_max"]["predicted"].flatten(), 
                         mode='lines+markers', name='Predicción transf_encoder_min_max'))

fig.add_trace(go.Scatter(x=time_data, y =constant_value*np.ones(5)))


# Update the layout for clarity
fig.update_layout(title='Predicción de precios de cierre de Bitcoin',
                  xaxis_title='Tiempo',
                  yaxis_title='Precio de cierre')


# Display the plot
fig.show()

# <font color="8C3061" >**Conclusión**</font> 

> - Entre los modelos implementados el que tuvo mejor desempeño fue el Encoder con datos normalizados con MinMaxEscaler. Este modelo pudo describir de mejor manera el comportamiento de la moneda en la inferencia analizada.
> - El modelo Seq2Seq tuvo en la inferencia analizada tuvo mejor desempeño.
> - El modelo con normalización logaritmica fue el que tuvo peor desempeño por mucho.
> - El modelo Seq2Seq no le ganó al modelo que mantiene su predicción constante.

In [27]:
s2s = inference_dict['seq2seq']["predicted"].flatten()
cons = constant_value*np.ones(5)
real = inference_dict['transf_encoder_zscore']["true"].flatten()

print("MSE seq2seq:\t ", mean_squared_error(real, s2s))
print("MSE constante:\t ", mean_squared_error(real, cons))

MSE seq2seq:	  92755.64125570907
MSE constante:	  38551.151625536106
